In [14]:
import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms
import os

# Initialize LPIPS model
loss_fn = lpips.LPIPS(net='alex')

# Function to load and preprocess image
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    img = transform(img)
    return img.unsqueeze(0)

# Paths to the directories containing reference and generated images
ref_images_dir = '../full-version/flowers_1000train_10test_epochs=100_steps=200/'
gen_images_dir = './2_color/'

# Get list of image filenames
ref_images = sorted(os.listdir(ref_images_dir))
gen_images = sorted(os.listdir(gen_images_dir))

# Ensure both directories contain the same number of images
assert len(ref_images) == len(gen_images), "Directories do not contain the same number of images"

# Calculate LPIPS score for each pair of images
lpips_scores = []
for ref_img, gen_img in zip(ref_images, gen_images):
    img1 = load_image(os.path.join(ref_images_dir, ref_img))
    img2 = load_image(os.path.join(gen_images_dir, gen_img))
    
    with torch.no_grad():
        score = loss_fn(img1, img2) # SA VAD DE ASTA
    
    lpips_scores.append(score.item())

# Calculate the average LPIPS score
average_lpips = sum(lpips_scores) / len(lpips_scores)
print('Average LPIPS score:', average_lpips)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Cezica\miniconda3\envs\Multimedia\lib\site-packages\lpips\weights\v0.1\alex.pth
Average LPIPS score: 0.2756832897663116


In [15]:
import cv2
import os
import numpy as np

def calculate_psnr(original_image, result_image):
    # Load images
    img1 = cv2.imread(original_image)
    img2 = cv2.imread(result_image)

    # Convert to float64
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)

    # Calculate MSE (Mean Squared Error)
    mse = np.mean((img1 - img2) ** 2)

    if mse == 0:
        return float('inf')  # PSNR is infinite if images are identical

    # Calculate PSNR
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))

    return psnr

def calculate_average_psnr(original_folder, results_folder):
    original_images = sorted(os.listdir(original_folder))
    result_images = sorted(os.listdir(results_folder))

    assert len(original_images) == len(result_images), "Number of original and result images must be the same"
    
    total_psnr = 0.0
    num_pairs = len(original_images)
    
    for orig_img, res_img in zip(original_images, result_images):
        orig_img_path = os.path.join(original_folder, orig_img)
        res_img_path = os.path.join(results_folder, res_img)
        psnr = calculate_psnr(orig_img_path, res_img_path)
        total_psnr += psnr
    
    average_psnr = total_psnr / num_pairs
    
    return average_psnr

# Exemplu de folosire
original_folder = '../full-version/flowers_1000train_10test_epochs=100_steps=200/'
results_folder = './2_color/'  # Aici se presupune că ai salvat imaginile colorizate într-un folder numit 'result'

average_psnr = calculate_average_psnr(original_folder, results_folder)

print(f"Average PSNR across all pairs: {average_psnr:.2f} dB")

Average PSNR across all pairs: 17.80 dB


In [16]:
import os
from skimage.metrics import structural_similarity as ssim
from skimage import io
import numpy as np

def calculate_ssim(img1_path, img2_path):
    # Încarcă imaginile și convertește-le la float32
    img1 = io.imread(img1_path, as_gray=True).astype(np.float32)
    img2 = io.imread(img2_path, as_gray=True).astype(np.float32)

    # Calculează SSIM
    ssim_index, _ = ssim(img1, img2, full=True, data_range=1.0)
    return ssim_index

def calculate_ssim_between_folders(folder1_path, folder2_path):
    # Lista de imagini din fiecare folder
    imgs_folder1 = sorted(os.listdir(folder1_path))
    imgs_folder2 = sorted(os.listdir(folder2_path))

    # Verificăm dacă folderele au același număr de imagini
    if len(imgs_folder1) != len(imgs_folder2):
        print("Folderele nu conțin același număr de imagini. Nu se poate efectua compararea.")
        return

    ssim_scores = []
    for img1_name, img2_name in zip(imgs_folder1, imgs_folder2):
        img1_path = os.path.join(folder1_path, img1_name)
        img2_path = os.path.join(folder2_path, img2_name)

        ssim_score = calculate_ssim(img1_path, img2_path)
        ssim_scores.append(ssim_score)

        print(f"SSIM Score pentru {img1_name} și {img2_name}: {ssim_score:.4f}")

    # Calculăm scorul mediu de SSIM
    mean_ssim = sum(ssim_scores) / len(ssim_scores)
    print(f"Scorul mediu de SSIM pentru toate perechile: {mean_ssim:.4f}")

# Exemplu de folosire
folder1_path = '../full-version/flowers_1000train_10test_epochs=100_steps=200/'
folder2_path = './2_color/'

calculate_ssim_between_folders(folder1_path, folder2_path)


SSIM Score pentru img_0.png și img_0.png: 0.9827
SSIM Score pentru img_1.png și img_1.png: 0.9818
SSIM Score pentru img_2.png și img_2.png: 0.9997
SSIM Score pentru img_3.png și img_3.png: 0.9970
SSIM Score pentru img_4.png și img_4.png: 0.9681
SSIM Score pentru img_5.png și img_5.png: 0.9815
SSIM Score pentru img_6.png și img_6.png: 0.9910
SSIM Score pentru img_7.png și img_7.png: 0.9681
SSIM Score pentru img_8.png și img_8.png: 0.9774
SSIM Score pentru img_9.png și img_9.png: 0.9910
Scorul mediu de SSIM pentru toate perechile: 0.9838
